In [1]:
import keras.metrics
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras import models, layers
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Reshape, BatchNormalization, LeakyReLU
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import keras
import tensorflow as tf
import os
from sklearn.utils import class_weight

/Users/helenamehler/PycharmProjects/fallDetection/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def windowing(x, y, window_size, overlap):
    windows = []
    y_labels = []

    step_size = window_size - overlap

    for i in range(0, len(x) - window_size + 1, step_size):
        window = x.iloc[i:(i + window_size), :]
        windows.append(window)

        if np.sum(y[i:i + window_size] == 1) >= 3:
            y_labels.append(1)
        else:
            y_labels.append(0)

    return np.array(windows), np.array(y_labels)

In [4]:
data_path = '../../dataSets/SisFall_labelled'
dfs = []

for root, dirs, files in os.walk(data_path):
    for file in files:
        file_path = os.path.join(root, file)
        dfs.append(pd.read_csv(file_path, index_col=0))

In [170]:
window_size = 200
overlap = 100

In [171]:
windowed_x = []
windowed_y = []
for i, dataset in enumerate(dfs):
    x_w, y_w = windowing(dataset.drop(columns = ['fall']), dataset['fall'], window_size, overlap)
    windowed_x.extend(x_w)
    windowed_y.extend(y_w)

In [248]:
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(len(windowed_x)):
    if np.random.rand(1)[0] < 0.2:
        x_test.append(windowed_x[i])
        y_test.append(windowed_y[i])
    else:
        x_train.append(windowed_x[i])

        y_train.append(windowed_y[i])

In [249]:
scaler = MinMaxScaler()

x_train = np.array(x_train)
x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)

x_test = np.array(x_test)
x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [250]:
y_trainCat = to_categorical(y_train, num_classes=2)
input_shape = (200, 3)

In [282]:
class_weights = class_weight.compute_class_weight('balanced', classes = np.unique(y_train),y = y_train)
weight = {i : class_weights[i] for i in range(len(class_weights))}
#weight = {0: 1, 1:10}

trying out different models

In [179]:
model = models.Sequential()
model.add(Conv1D(filters=64, kernel_size=4, activation='relu', input_shape=(128,3)))
model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [271]:
model = models.Sequential()

# Convolutional layer
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(128, 3)))
model.add(MaxPooling1D(pool_size=2))

# LSTM layer
model.add(LSTM(20, activation='relu'))

# Dense layer for classification
model.add(Dense(2, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#other metric: metrics=[tf.keras.metrics.FalseNegatives(),tf.keras.metrics.FalsePositives(),'accuracy'] 

In [284]:
batch = 80
epochs = 40
iterationsPerEp = x_train.shape[0] // batch
model.fit(x_train, y_trainCat, batch_size=batch, epochs=epochs, steps_per_epoch=iterationsPerEp, class_weight = weight)

Epoch 1/40
2400/2400 [==============================] - 40s 16ms/step - loss: 0.8064 - accuracy: 0.7509
Epoch 2/40
2400/2400 [==============================] - 38s 16ms/step - loss: 0.7630 - accuracy: 0.7648
Epoch 3/40
2400/2400 [==============================] - 37s 15ms/step - loss: 0.7526 - accuracy: 0.7542
Epoch 4/40
2400/2400 [==============================] - 37s 15ms/step - loss: 0.7507 - accuracy: 0.7977
Epoch 5/40
2400/2400 [==============================] - 38s 16ms/step - loss: 0.7199 - accuracy: 0.8546
Epoch 6/40
2400/2400 [==============================] - 37s 15ms/step - loss: 0.6955 - accuracy: 0.8827
Epoch 7/40
2400/2400 [==============================] - 39s 16ms/step - loss: 0.6648 - accuracy: 0.9017
Epoch 8/40
2400/2400 [==============================] - 37s 16ms/step - loss: 0.6251 - accuracy: 0.9205
Epoch 9/40
2400/2400 [==============================] - 37s 16ms/step - loss: 0.6068 - accuracy: 0.9214
Epoch 10/40
2400/2400 [==============================] - 38s 16m

In [285]:
predictions = model.predict(x_test)
yPredClasses = np.argmax(predictions, axis=1)
confMat = confusion_matrix(y_test, yPredClasses)
confMat

1493/1493 [==============================] - 19s 12ms/step


array([[43401,  2016],
       [ 1074,  1273]])

In [286]:
print('Accuracy', accuracy_score(y_test, yPredClasses))
print('F1-Score', f1_score(y_test, yPredClasses))
print('AUC', roc_auc_score(y_test, yPredClasses))

Accuracy 0.935306925718114
F1-Score 0.4517388218594748
AUC 0.749002940595954
